In [51]:
import pysentiment2 as ps
import pandas as pd 
import numpy as np 
import glob, os
import sys 
utils_path = os.path.expanduser("~/Documents/Documents - KBO’s MacBook Pro/Utils/")
sys.path.insert(0, utils_path)
from Plotly_Utils import pplot, pplot_df, heatmap
root = '/Users/rayboppana/Documents/Documents - KBO’s MacBook Pro/MSCF/Mini_5/NLP/HW1/the_tens/'

import nltk
from nltk import word_tokenize
from nltk import FreqDist
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('omw-1.4')    
nltk.download('wordnet')
nltk.download('stopwords')

from tqdm import tqdm 
from collections import Counter 
import matplotlib.pyplot as plt 
import re
os.chdir('/Users/rayboppana/Documents/Documents - KBO’s MacBook Pro/MSCF/Mini_5/NLP/HW1/')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rayboppana/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/rayboppana/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rayboppana/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rayboppana/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [42]:
# Harvard and Loughron/McDonald Dictionaries
hiv4 = ps.HIV4()
lm = ps.LM()

dicts = [hiv4,lm]
def get_text_score(text, dictionary):
    tokens = dictionary.tokenize(text)
    return dictionary.get_score(tokens)

text = 'Tax'
cols = get_text_score(text,hiv4).keys()
scores = [list(get_text_score(text,d).values()) for d in dicts]
pd.DataFrame(scores, columns=cols, index=['Harvard', 'Fin-Neg'])



,Positive,Negative,Polarity,Subjectivity
Harvard,0,1,-0.999999,0.999999
Fin-Neg,0,0,0.000000,0.000000


In [43]:
mkt_returns = pd.read_csv('mkt_returns.csv')
mkt_returns['caldt'] = pd.to_datetime(mkt_returns['caldt'], format='%Y%m%d')
mkt_returns.set_index('caldt', inplace=True)
mkt_returns = mkt_returns.rolling(4).sum().shift(-4)
mkt_returns

,vwretd
caldt,
2011-01-03,0.000385
2011-01-04,0.000145
2011-01-05,-0.001190
2011-01-06,0.009557
2011-01-07,0.009615
...,...
2021-12-27,-0.005258
2021-12-28,NaN
2021-12-29,NaN


In [44]:
filename = 'stock_daily_gvkey.csv'
date_col = 'datadate'
id_col = 'GVKEY'
price_col = 'prccd'

stock_data = pd.read_csv(filename, low_memory=False)
stock_data[date_col] = pd.to_datetime(stock_data[date_col], format='%Y%m%d')
stock_data = stock_data.drop_duplicates(subset=[id_col,date_col])
stock_data.set_index([id_col, date_col], inplace=True)
stock_data = stock_data[price_col]
price_data = stock_data.unstack(level=0)
price_data

GVKEY,1045,1075,1078,1161,1177,1209,1230,1279,1300,1327,...,187450,187697,188255,189491,198058,199356,260774,260778,294524,316056
datadate,,,,,,,,,,,,,,,,,,,,,
2010-01-04,7.72,36.88,54.46,9.70,33.00,83.16,34.87,23.52,40.35,14.82,...,NaN,NaN,NaN,NaN,NaN,NaN,13.82,37.15,NaN,NaN
2010-01-05,8.36,36.38,54.02,9.71,32.53,82.47,36.78,23.13,40.43,15.05,...,NaN,NaN,NaN,NaN,NaN,NaN,14.15,37.10,NaN,NaN
2010-01-06,8.11,36.76,54.32,9.57,32.40,81.79,36.45,23.66,40.43,14.91,...,NaN,NaN,NaN,NaN,NaN,NaN,14.03,36.67,NaN,NaN
2010-01-07,8.25,36.69,54.77,9.47,33.43,81.32,36.26,23.21,41.21,14.74,...,NaN,NaN,NaN,NaN,NaN,NaN,14.17,37.18,NaN,NaN
2010-01-08,8.11,36.92,55.05,9.43,32.70,81.84,35.54,22.99,42.12,15.00,...,NaN,NaN,NaN,NaN,NaN,NaN,14.20,36.40,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,18.17,68.89,141.46,154.36,NaN,302.22,52.62,NaN,206.43,155.36,...,188.69,697.00,104.76,118.29,63.36,27.24,107.79,NaN,91.67,129.58
2021-12-28,18.54,69.72,140.47,153.15,NaN,303.99,52.90,NaN,207.05,155.94,...,186.79,682.20,105.91,119.52,63.04,27.37,107.72,NaN,92.72,131.06
2021-12-29,18.05,69.87,141.19,148.26,NaN,305.40,52.14,NaN,207.53,156.37,...,185.47,677.84,106.63,119.36,64.01,27.21,108.58,NaN,92.67,132.37


In [45]:
excess_returns = price_data.shift(-4)/price_data-1
excess_returns.loc[mkt_returns.index] -= np.array(mkt_returns)
excess_returns

GVKEY,1045,1075,1078,1161,1177,1209,1230,1279,1300,1327,...,187450,187697,188255,189491,198058,199356,260774,260778,294524,316056
datadate,,,,,,,,,,,,,,,,,,,,,
2010-01-04,0.050518,0.001085,0.010834,-0.027835,-0.009091,-0.015873,0.019214,-0.022534,0.043866,0.012146,...,NaN,NaN,NaN,NaN,NaN,NaN,0.027496,-0.020188,NaN,NaN
2010-01-05,-0.037081,0.031336,0.024250,-0.058702,0.006456,0.010549,-0.039695,-0.003891,0.054910,-0.002658,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.006360,-0.040162,NaN,NaN
2010-01-06,-0.036991,0.018770,0.015648,-0.096134,-0.055247,0.005991,-0.009328,-0.030008,0.043037,-0.013414,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.032074,-0.047450,NaN,NaN
2010-01-07,-0.019394,0.026165,0.009494,-0.033791,-0.080168,0.021889,0.029785,-0.011202,0.026935,0.021031,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.015526,-0.038999,NaN,NaN
2010-01-08,0.022195,0.021939,0.005450,-0.045599,-0.039450,0.002810,0.052335,-0.011744,0.010684,0.022000,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.050704,-0.011538,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,-0.006300,0.029935,0.000168,-0.062506,NaN,0.012008,-0.004624,NaN,0.015334,0.003842,...,-0.025215,-0.035703,0.025686,0.019038,0.029721,0.005992,0.011938,NaN,0.011367,0.027329
2021-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
ignore = ['.DS_Store', '.ipynb_checkpoints']
os.chdir('/Users/rayboppana/Documents/Documents - KBO’s MacBook Pro/MSCF/Mini_5/NLP/HW1/')
folders = os.listdir()

# Harvard and Loughron/McDonald Dictionaries
hiv4 = ps.HIV4()
lm = ps.LM()

dicts = [hiv4,lm]
def get_text_score(text, dictionary):
    tokens = dictionary.tokenize(text)
    return dictionary.get_score(tokens)

# text preprocessing
def lemmatize_words(words):
    lemmatized_words = [WordNetLemmatizer().lemmatize(word, 'v') for word in words]
    return lemmatized_words
    
def preprocess_text(text):
    # 1. switch to lower case
    clean_text = text.lower()

    # 2. lemmatize the data
    word_pattern = re.compile('\w+')
    text_lemma = lemmatize_words(word_pattern.findall(clean_text))
    
    # 3. remove the stop words
    lemma_english_stopwords = lemmatize_words(stopwords.words('english'))
    text_lemma = [word for word in text_lemma if word not in lemma_english_stopwords]
    
    # preprocessed text
    lemma_doc =  ' '.join(text_lemma)
    
    return lemma_doc

  
# Get all folders 
subfolders = [x[0] for x in os.walk(root )][1:]
gvkey_txt_dict = dict() # key = gvkey, value = name of txt

N = 0
gvk_txt = []
for folder in subfolders:
    os.chdir(folder)
    txt_names=[]
    for file in glob.glob("*.txt"):
        txt_names.append(folder.split('/')[-1] + '/'+ file)
        N+=1
    gvk_txt.extend(txt_names)

# compute M, alpha vector, and excess returns vector in first pass 
all_tokens = []
alpha = np.zeros(N)
excess_returns_vec = np.zeros(N)
alpha_index = 0
print("Computing M, alpha, and excess returns")
texts = []
for file in tqdm(gvk_txt):
    with open(root + file,'r') as f:
        text = f.read()
        text = preprocess_text(text)
        texts.append(text)
        
    tokens = word_tokenize(text)
    alpha[alpha_index] = len(tokens)/len(set(tokens))

    all_tokens.extend(tokens)

    date = file[file.index('/')+1: file.index('.')]
    gvkey = int(file[:file.index('/')])
    if date in excess_returns.index and gvkey in excess_returns.columns:
        excess_returns_vec[alpha_index] = excess_returns.loc[date, gvkey]
    else:
        excess_returns_vec[alpha_index] = np.nan

    alpha_index += 1

unique_words = list(set(all_tokens))
M = len(unique_words)
word_index = dict(zip(unique_words, np.arange(M)))

# compute TF and df in second pass 
tf = np.zeros((M,N))
df = np.zeros(M)
j = 0
print("Computing TF and df")
for text in tqdm(texts):
    tokens = word_tokenize(text)
    count_dict = Counter(tokens)
    for word in count_dict:
        i = word_index[word]
        tf[i,j] = count_dict[word]
        df[i] += 1
    j += 1

# compute W matrix 
W = (1 + np.log(tf))*( np.log(N/df).reshape(-1,1) @ (1/(1+np.log(alpha)).reshape(1,-1)))
W[W==np.NINF] = 0

# Add two negativity_flag row vector from hiv and fin_neg
neg_hiv4= np.zeros((1,M))
neg_lm = np.zeros((1,M))

for i in range(len(unique_words)):
    neg_hiv4[0,i] = get_text_score(unique_words[i],hiv4)['Negative']
    neg_lm[0,i] =  get_text_score(unique_words[i],lm)['Negative']

# negative proportion for each document
neg_prop_hiv = neg_hiv4 @ W / W.sum()
neg_prop_finneg = neg_lm @ W/ W.sum()



results = pd.DataFrame({"Harvard Negative Proportion": neg_prop_hiv.reshape(-1),
                        "Fin Neg Negative Proportion": neg_prop_finneg.reshape(-1),
                        "Excess Returns": excess_returns_vec})




results

Computing M, alpha, and excess returns


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 88/88 [00:23<00:00,  3.79it/s]


Computing TF and df


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 88/88 [00:06<00:00, 13.24it/s]
/var/folders/zl/94x3_2q90pz6190q9z3x5_c00000gp/T/ipykernel_32557/532084717.py:102: RuntimeWarning:

divide by zero encountered in log



,Harvard Negative Proportion,Fin Neg Negative Proportion,Excess Returns
0,0.000491,0.000511,-0.016024
1,0.002010,0.002497,0.052218
2,0.000440,0.000463,-0.000250
3,0.000462,0.000476,0.006011
4,0.000286,0.000392,-0.015119
...,...,...,...
83,0.001104,0.001144,NaN
84,0.000329,0.000202,NaN
85,0.000323,0.000221,NaN
86,0.000259,0.000327,NaN


In [57]:
results_copy = results.copy()
results = results.dropna()
results['Harvard Quintile'] = pd.qcut(results['Harvard Negative Proportion'],5)
results['Fin Neg Quintile'] = pd.qcut(results['Fin Neg Negative Proportion'],5)

harvard_agg_excess_ret = results.groupby(by='Harvard Quintile').median()['Excess Returns']
harvard_agg_excess_ret.index = [1,2,3,4,5]

Fin_Neg_excess_ret = results.groupby(by='Fin Neg Quintile').median()['Excess Returns']
Fin_Neg_excess_ret.index = [1,2,3,4,5]

final_results = pd.concat([harvard_agg_excess_ret, Fin_Neg_excess_ret], axis=1)
final_results.columns=['H4N-Inf', 'Fin-Neg']
pplot_df(final_results, yaxes_labels=['Median Filing Period Excess Return'],
         title='Excess Return vs Quintile',
         x_label='Quintile (based on proportion of negative words)')

In [58]:
final_results

,H4N-Inf,Fin-Neg
1,-0.000250,-0.006771
2,-0.009552,-0.006084
3,0.004875,-0.015668
4,-0.019929,-0.016486
5,-0.021508,-0.021508
